In [ ]:
import calendar

import pandas as pd
import requests
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from tabula import read_pdf
import zipfile


In [ ]:
with zipfile.ZipFile("Sales.csv.zip","r") as zip_ref:
    zip_ref.extractall()

In [ ]:
df = pd.read_csv('Sales.csv')

In [ ]:
df.head()

In [ ]:
df.columns = ['id', 'date','zip','payment_usd','hospital_id','hospital_name']

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['date'].head()

In [ ]:
df['yr'] = pd.DatetimeIndex(df['date']).year 
df['month'] = pd.DatetimeIndex(df['date']).month

In [ ]:
df['year_month'] = df['yr'].astype(str) + '-' + df['month'].astype(str)

In [ ]:
df.head()

## Total Sales

In [ ]:
total_sales = df['payment_usd'].sum()

In [ ]:
'${:,.2f}'.format(total_sales) 

## Total Sales by Month

In [ ]:
df_sales_by_month = df.groupby(['yr', 'month'])['payment_usd'].sum()
monthly_sales = df_sales_by_month.reset_index()
monthly_sales

In [ ]:
[calendar.month_name[i] for i in range(1,13)]

In [ ]:
def line_plot(x, y, legend, title, x_label, y_label):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x = x,
        y=y,
        name=legend

    ))

    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title=y_label,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )


    return fig

In [ ]:
x = [calendar.month_name[i] for i in range(1,13)]
y=monthly_sales['payment_usd']
legend='Sales'
title="Total Montly Revenue"
x_label='Month'
y_label='Monthly Revenue (USD)'
fig = line_plot(x, y, legend, title, x_label, y_label)
fig.show()

In [ ]:
fig.write_html('results/fig1_monthly_revenue.html')

## Total Sales by Zip Code

In [ ]:
set([len(i) for i in df['zip'].astype('str')])

We have Nan, regular zips, and zip+4, need to collapse down to regular zip.  
NaNs are ignored on the groupby

In [ ]:
def trim_zip(zip):
    return zip[:5]

trim_zip('12345-1234')

In [ ]:
df['5_digit_zip'] = df['zip'].astype('str').apply(trim_zip)

In [ ]:
df_zip_sales = df.groupby(['5_digit_zip'])['payment_usd'].sum()
df_zip_sales = df_zip_sales.reset_index()
df_zip_sales.head()

It doesn't make a lot of sense to try and plot 12K entries in one chart so we'll do top ten and bottom ten

In [ ]:
df_top_ten = df_zip_sales.sort_values(['payment_usd'])[-10:]
df_bottom_ten = df_zip_sales.sort_values(['payment_usd'])[:10]

In [ ]:
def pie_plot(labels, values, title):
    fig = go.Figure()

    fig.add_trace(
        go.Pie(
            labels = labels,
            values = values,
    ))

    fig.update_layout(
        title = title,
        font=dict(
            family="Courier New, monospace",
            size=18,
            color="#7f7f7f"
        )
    )


    return fig

In [ ]:
labels = df_top_ten['5_digit_zip'].astype('str')
values = df_top_ten['payment_usd']
title="Top 10 Revenue Sources by Zip: 2018"
fig = pie_plot(labels, values, title)
fig.show()

In [ ]:
fig.write_html('results/fig2_Top_10_Revenue_by_zip.html')

Hover over pie slice to see absolute value

In [ ]:
labels = df_bottom_ten['5_digit_zip'].astype('str')
values = df_bottom_ten['payment_usd']
title="Bottom 10 Revenue Sources by Zip: 2018"
fig = pie_plot(labels, values, title)
fig.show()

In [ ]:
fig.write_html('results/fig3_Bottom_10_Revenue_by_zip.html')

Hover over pie slice to see absolute value

## Breakdown of sales by teaching hospital vs non-teaching hospital

There were no columns in the dataset referring to the teaching hospital status so I found a 3rd party dataset in pdf format. The below code downloads that pdf, processes it, and left joins back to the original dataframe in order to categorize each hosptial.

In [ ]:
teach_hosp_url ='https://www.cms.gov/OpenPayments/Downloads/2018-Reporting-Cycle-Teaching-Hospital-List-pdf.pdf'

In [ ]:
r = requests.get(teach_hosp_url)

Download a pdf of the teaching hospitals as of Oct 2018

In [ ]:
with open('teaching_hospitals.pdf', 'wb') as f:
    f.write(r.content)

In [ ]:
df_teach_hosp = read_pdf('teaching_hospitals.pdf', pages='all', pandas_options={'header': None})

Massage into single data frame:  
Note: the hospital id fields did not match those in the above pdf so had to use the name and zip as our join fields

The PDF Scraper parsed the tables inconsistenly on different pages. I had to manually determine which column numbers correspond to which fields   
9 cols, name=1, zip=8  
12 cols, name=2, zip=7  
13 cols, name =2, zip=8  
14 cols, name=2, zip=8  

In [ ]:
parse_dict = {9:(1,8), 12:(2,7), 13:(2,8), 14:(2, 8)}

In [ ]:
name_list = []
zip_list = []
for df_page in df_teach_hosp:
    num_cols = len(df_page.columns)
    name_col, zip_col = parse_dict[num_cols]
    name_list.append(df_page.iloc[:, name_col].tolist())
    zip_list.append(df_page.iloc[:, zip_col].tolist())

In [ ]:
def list_flatten(input_list):
    # Flatten List of lists
    flat_list = []
    for sublist in input_list:
        for item in sublist:
            flat_list.append(item)
    return flat_list

In [ ]:
# first two entries on page 1 are nan and header
flat_name_list = list_flatten(name_list)[2:]
flat_zipcode_list = list_flatten(zip_list)[2:]

In [ ]:
df_teaching_hosp = pd.DataFrame(flat_name_list, flat_zipcode_list)

In [ ]:
df_teaching_hosp.columns = ['teaching_hosp']

In [ ]:
df_teaching_hosp['zip_code'] = df_teaching_hosp.index

In [ ]:
df_teaching_hosp.head()

In [ ]:
# need zip to be str type for join later
df['zip'] = df['zip'].astype('str')

In [ ]:
df_teaching_hosp.head()

Does our teaching_hosp data set use zip5?

In [ ]:
df_teaching_hosp['zip_code'].head()

In [ ]:
df['5_digit_zip'] = df['zip'].astype('str').apply(trim_zip)

In [ ]:
df_teaching_hosp['5_digit_zip'] = df_teaching_hosp['zip_code'].astype('str').apply(trim_zip)
#df_teaching_hosp['zip_code'].astype('str')

In [ ]:
# need zip to be str type for join later
#df_teaching_hosp['zip_code'] = df_teaching_hosp['zip_code'].astype('str')

In [ ]:
df_teaching_hosp.head()

In [ ]:
df.head()

In [ ]:
# Left join on zipcode and hospital name since hospital ID doesn't seem to match our 3rd party data source
joined_df = pd.merge(df, df_teaching_hosp,  how='left', left_on=['hospital_name','5_digit_zip'], right_on = ['teaching_hosp','5_digit_zip'])

#Join just by Hospital Name
#joined_df = pd.merge(df, df_teaching_hosp,  how='left', left_on=['hospital_name'], right_on = ['teaching_hosp'])


In [ ]:
# if we have a label in the joined col then teaching hospital, if na then not teaching hospital
joined_df['category_teaching'] = ~joined_df['teaching_hosp'].isna()

In [ ]:
joined_df.head()

In [ ]:
df_teaching_schools = joined_df.groupby(['category_teaching'])['payment_usd'].sum()
df_teaching_schools = df_teaching_schools.reset_index()
df_teaching_schools[df_teaching_schools['category_teaching'] == True]

In [ ]:
df_teaching_schools = df_teaching_schools.replace([True, False], ['teaching', 'non-teaching'])

In [ ]:
labels = df_teaching_schools['category_teaching'].astype('str')
values = df_teaching_schools['payment_usd']
title="2018 Revenue Breakdown by Teaching Institution (USD)"
fig = pie_plot(labels, values, title)
fig.show()
fig.write_html('results/fig4_Revenue_by_Teaching_vs_Non_Teach_Inst.html')

It appears ~ 16% of revenue is from Teaching Institutions

### Notes
This 16% seems low so I also tried join just on the name and excluded zip and the percent of revenue of teaching institutions rose to 23.1%. However there are many generic hospital names like Saint Mary's so using a zip in the join ensures we don't over count teaching institutions. The percentage is likely to be between 16 - 23%.